In [1]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [8]:
# Standard library imports
import os
import re
import sys
import json
import base64
from io import BytesIO

# Other library imports
import boto3
import numpy as np
from PIL import Image

# Print SDK versions
print(f"Python version: {sys.version.split()[0]}")
print(f"Boto3 SDK version: {boto3.__version__}")
print(f"NumPy version: {np.__version__}")

# Try to import scipy with error handling
try:
    from scipy.spatial.distance import cdist
    print("SciPy imported successfully")
except Exception as e:
    print(f"SciPy import failed: {e}")
    print("Using NumPy fallback for distance calculations")
    
    # Simple numpy-based distance calculation
    def cdist_numpy(XA, XB, metric='euclidean'):
        """Simple numpy-based distance calculation as fallback"""
        if metric == 'euclidean':
            return np.sqrt(((XA[:, np.newaxis, :] - XB[np.newaxis, :, :]) ** 2).sum(axis=2))
        else:
            raise ValueError(f"Metric '{metric}' not implemented in fallback")
    
    cdist = cdist_numpy

# Try to import seaborn with error handling
try:
    import seaborn as sns
    print("Seaborn imported successfully")
except Exception as e:
    print(f"Seaborn import failed: {e}")
    print("Continuing without seaborn - matplotlib will be used instead")
    sns = None
    

Python version: 3.12.2
Boto3 SDK version: 1.36.1
NumPy version: 1.26.4
SciPy imported successfully
Seaborn imported successfully
